In [117]:
import pandas as pd
import dateutil.parser
from datetime import date
import re


In [139]:
detention_stats = pd.read_csv("https://raw.githubusercontent.com/haydenbetts/ny-pretrial-detention/main/Daily_Incarcerated_In_Custody_NYC_11_28_21.csv")
penal_code = pd.read_csv("https://raw.githubusercontent.com/haydenbetts/ny-pretrial-detention/main/ny-penal-code.csv")
penal_code_secondary = pd.read_csv("https://raw.githubusercontent.com/andyj1/bailproject/master/src/cleaned/newyork.csv")


In [144]:
# Merge Penal Code Data Source #1
penal_code["TOP_CHARGE"] = penal_code["CODE"]
merged = pd.merge(detention_stats, penal_code, how="left", on=["TOP_CHARGE"])
merged["DAYS_SINCE_ADMISSION"] = merged["ADMITTED_DT"].map(lambda d: f'{date.today() - dateutil.parser.parse(d).date()}'[:-14])
merged.sort_values(by=['DAYS_SINCE_ADMISSION'], ascending=False)

# Clean Penal Code Data Source #2
# Penal Law OFFENSE,CLASS

def search(string, capture_index):
    regex = "[0-9]+\s+([\w|\s|-]+) ([A-F]{1} Felony)"
    match = re.search(regex, string)
    if match is None:
        return ""
    else:
        print(match)
        return match[capture_index]

penal_code_secondary["TOP_CHARGE"] = penal_code_secondary["charges"].map(lambda c: c.split(" ")[0]) 
penal_code_secondary.drop_duplicates(subset=['TOP_CHARGE'], inplace=True)
penal_code_secondary["Penal Law OFFENSE"] = penal_code_secondary["charges"].map(lambda c: search(c, 1))
penal_code_secondary['CLASS'] = penal_code_secondary["charges"].map(lambda c: search(c, 2))
# ([A-F]{1} Felony)

penal_code_secondary.head()

# Merge Data from Penal Code Source #2
# def new_charge_if_exists(n):
#     if not pd.isna(n["Penal Law OFFENSE"]):
#         return n
#     else:
#         r = penal_code_secondary.loc[penal_code_secondary['TOP_CHARGE'] == n["TOP_CHARGE"]]
#         if not r.empty:
#             n["Penal Law OFFENSE"] = r["Penal Law OFFENSE"].to_string()
#             n["CLASS"] = r["CLASS"].to_string()
#         return n
    
# merged = merged.apply(lambda c: new_charge_if_exists(c), axis=1)

# merged.to_csv('results.csv', index=False, header=True)


<re.Match object; span=(33, 51), match='1 MILLION B Felony'>
<re.Match object; span=(33, 48), match='18 YRS A Felony'>
<re.Match object; span=(33, 51), match='1 MILLION B Felony'>
<re.Match object; span=(33, 48), match='18 YRS A Felony'>


,bond_info,charges,race,gender,first_statute,TOP_CHARGE,Penal Law OFFENSE,CLASS
0,50000.0,110-120.10 FC (Attempted ASSAULT-1ST C Felony),Other,Male,110120,110-120.10,,
1,50000.0,110-125.25 FB (Attempted MURDER B Felony),Black,Male,110125,110-125.25,,
3,15000.0,120.05 FD (ASSAULT -2ND D Felony),Black,Male,12005,120.05,,
4,500000.0,230.34 FB ( B Felony),Black,Male,23034,230.34,,
5,200000.0,140.30 FB (BURGLARY-1ST B Felony) |160.10 FC (...,Black,Male,14030,140.30,,
